# Project 3: Predicting new Airbnb users' first destinations
# Part 2: SQL and EDA

## Using SQL to perform some EDA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from patsy import dmatrix
import datetime as dt
import numpy as np
import pickle

%matplotlib inline

from sqlalchemy import create_engine
cnx = create_engine('postgresql://emilygeller:p@54.173.47.58:5432/emilygeller')

/Users/emilygeller/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
pd.read_sql('select * from countries;', cnx)

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


In [3]:
pd.read_sql('select * from sessions limit 5;', cnx)

,id,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,0,d1mm9tcy42,lookup,None,None,Windows Desktop,319.0
1,1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,2,d1mm9tcy42,lookup,None,None,Windows Desktop,301.0
3,3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,4,d1mm9tcy42,lookup,None,None,Windows Desktop,435.0


## Constructing features from Sessions data
I'll use SQL to build them.

### Total sessions per user

In [4]:
q = '''
select user_id, count(1)
from sessions
group by user_id
limit 5;'''

In [5]:
pd.read_sql(q,cnx)

,user_id,count
0,00023iyk9l,40
1,0010k6l0om,63
2,001wyh0pz8,90
3,0028jgx1x1,31
4,002qnbzfs5,789


In [6]:
q = '''
select user_id, count(1)
from sessions
group by user_id
;'''

In [7]:
total_sessions = pd.read_sql(q,cnx)

### Total sessions per user per device

In [8]:
q = '''
select sessions.user_id, sessions.total as Mac
from (
    select user_id, device_type, count(1) total
    from sessions
    where device_type = 'Mac Desktop'
    group by user_id, device_type) sessions

    ;'''

In [9]:
sessions_per_device = pd.read_sql(q,cnx)

In [10]:
q = '''
select sessions.user_id, sessions.total as Windows
from (
    select user_id, device_type, count(1) total
    from sessions
    where device_type = 'Windows Desktop'
    group by user_id, device_type) sessions

    ;'''

In [11]:
sessions_per_device = sessions_per_device.merge(pd.read_sql(q,cnx), on = 'user_id', how = 'outer')

In [12]:
q = '''
select sessions.user_id, sessions.total as iPhone
from (
    select user_id, device_type, count(1) total
    from sessions
    where device_type = 'iPhone'
    group by user_id, device_type
    ) sessions

    ;'''

In [13]:
sessions_per_device = sessions_per_device.merge(pd.read_sql(q,cnx), on = 'user_id', how = 'outer')

In [14]:
q = '''
select sessions.user_id, sessions.total as Android
from (
    select user_id, device_type, count(1) total
    from sessions
    where device_type = 'Android'
    group by user_id, device_type
    ) sessions

    ;'''

In [15]:
sessions_per_device = sessions_per_device.merge(pd.read_sql(q,cnx), on = 'user_id', how = 'outer')

In [16]:
sessions_per_device = sessions_per_device.fillna(0)

In [17]:
sessions_per_device.head()

,user_id,mac,windows,iphone,android
0,00023iyk9l,36.0,0.0,4.0,0
1,0010k6l0om,63.0,0.0,0.0,0
2,0035hobuyj,489.0,0.0,0.0,0
3,00378ocvlh,75.0,0.0,0.0,0
4,0063bawn05,38.0,0.0,0.0,0


In [18]:
sessions_per_device.columns = ['id','total_sessions_on_mac','total_sessions_on_windows','total_sessions_on_iphone','total_sessions_on_android']

###  Total seconds on site per user

In [19]:
q = '''
select user_id, sum(secs_elapsed)
from sessions
group by user_id
limit 5;
'''

In [20]:
pd.read_sql(q,cnx)

,user_id,sum
0,00023iyk9l,867896.0
1,0010k6l0om,586543.0
2,001wyh0pz8,282965.0
3,0028jgx1x1,297010.0
4,002qnbzfs5,6487080.0


### Total seconds on site per user per device

In [21]:
q = '''
select device_type, count(user_id)
from sessions
group by device_type
order by count desc;'''

In [22]:
pd.read_sql(q,cnx)

,device_type,count
0,Mac Desktop,3585886
1,Windows Desktop,2648521
2,iPhone,2096749
3,Android Phone,835991
4,iPad Tablet,681836
5,Android App Unknown Phone/Tablet,272820
6,-unknown-,210059
7,Tablet,139859
8,Linux Desktop,27968
9,Chromebook,22272


Let's only look at the total time per user for the top four devices: `Mac Desktop`, `Windows Desktop`, `iPhone`, `Android Phone`.

In [23]:
q = '''
select sessions.user_id, sessions.total as Mac
from (
    select user_id, device_type, sum(secs_elapsed) total
    from sessions
    where device_type = 'Mac Desktop'
    group by user_id, device_type
    limit 5) sessions

    ;'''

In [24]:
pd.read_sql(q,cnx)

,user_id,mac
0,00023iyk9l,295642.0
1,0010k6l0om,586543.0
2,0035hobuyj,5724670.0
3,00378ocvlh,1827164.0
4,0063bawn05,2978016.0


Verification:

In [25]:
q = '''
select user_id, device_type, sum(secs_elapsed)
from sessions
where user_id in ('0010k6l0om', '0035hobuyj', '00023iyk9l')
group by user_id, device_type;'''

In [26]:
pd.read_sql(q,cnx)

,user_id,device_type,sum
0,00023iyk9l,iPhone,572254.0
1,0010k6l0om,Mac Desktop,586543.0
2,00023iyk9l,Mac Desktop,295642.0
3,0035hobuyj,Mac Desktop,5724670.0


## Feature Engineering

In [27]:
sessions = pd.read_csv('sessions_cleaned.csv')

In [28]:
users = pd.read_csv('users_cleaned.csv')

In [29]:
users['date_account_created'] = pd.to_datetime(users['date_account_created'], infer_datetime_format = True)
users['timestamp_first_active'] = pd.to_datetime(users['timestamp_first_active'], format = '%Y-%m-%d %H:%M:%S')
users['date_first_booking'] = pd.to_datetime(users['date_first_booking'],format = '%Y-%m-%d')

In [30]:
#users_dummied = pd.get_dummies(users[['gender','signup_method','signup_app','first_device_type','first_browser','affiliate_channel','affiliate_provider','first_affiliate_tracked']],drop_first = True)
users_dummied = pd.get_dummies(users[['gender','signup_method','signup_app','first_device_type','first_browser','affiliate_channel','affiliate_provider','first_affiliate_tracked']],drop_first = True)

In [31]:
users_dummied['langues_en'] = pd.get_dummies(users['language']).loc[:,'en']

Find the length of time between the account being created and the user being first active.

Positive -> user was active before they made their account

-1 -> First time the user was active was when they made their account

In [32]:
users_dummied['diff_account_to_first_active'] = (users['date_account_created']-users['timestamp_first_active']).dt.days
users_dummied['diff_account_to_first_booking'] = (users['date_first_booking']-users['date_account_created']).dt.days

In [33]:
users_dummied

,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_facebook,signup_method_google,signup_app_Moweb,signup_app_Web,signup_app_iOS,first_device_type_Android Tablet,first_device_type_Desktop (Other),...,affiliate_provider_yandex,first_affiliate_tracked_local ops,first_affiliate_tracked_marketing,first_affiliate_tracked_omg,first_affiliate_tracked_product,first_affiliate_tracked_tracked-other,first_affiliate_tracked_untracked,langues_en,diff_account_to_first_active,diff_account_to_first_booking
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,475,-57
1,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,764,278
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,279,-208
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,-1,1
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,-1,3
5,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,-1,10
6,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,-1,206
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,-1,0
8,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,-1,2
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,-1,13


In [34]:
users.language.value_counts()

en    86664
zh      521
fr      399
es      275
de      270
ko      241
ru      110
it      104
ja       84
pt       62
sv       44
nl       35
tr       20
da       19
pl       13
no       13
cs        9
fi        7
el        6
hu        4
th        4
is        2
ca        2
Name: language, dtype: int64

In [35]:
users_dummied.head()

,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_facebook,signup_method_google,signup_app_Moweb,signup_app_Web,signup_app_iOS,first_device_type_Android Tablet,first_device_type_Desktop (Other),...,affiliate_provider_yandex,first_affiliate_tracked_local ops,first_affiliate_tracked_marketing,first_affiliate_tracked_omg,first_affiliate_tracked_product,first_affiliate_tracked_tracked-other,first_affiliate_tracked_untracked,langues_en,diff_account_to_first_active,diff_account_to_first_booking
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,475,-57
1,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,764,278
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,279,-208
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,-1,1
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,-1,3


In [36]:
users_dummied['id'] = users['id']
users_dummied['age'] = users['age']

Turn Target into binary classes (domestic v international)

In [38]:
users.country_destination

0           US
1        other
2           US
3           US
4           US
5           US
6           US
7           US
8           US
9           FR
10          CA
11          US
12          FR
13          US
14          US
15          US
16          FR
17          US
18          FR
19          US
20          FR
21          US
22          US
23          GB
24          US
25          ES
26          US
27          US
28          US
29          US
         ...  
88878       US
88879       US
88880       ES
88881       US
88882       NL
88883       GB
88884       US
88885       US
88886       US
88887       US
88888       US
88889       IT
88890       AU
88891       US
88892    other
88893       FR
88894       US
88895    other
88896       US
88897       US
88898       ES
88899       US
88900       US
88901       US
88902       US
88903       FR
88904       US
88905       ES
88906       US
88907       US
Name: country_destination, Length: 88908, dtype: object

In [ ]:
#country_destinations = pd.get_dummies(users['country_destination'])
#y = country_destinations['US']

In [39]:
total_sessions.columns = ['id', 'total_sessions']

In [40]:
users_dummied = pd.merge(users_dummied, total_sessions, on='id', how = 'left')


In [41]:
users_dummied = pd.merge(users_dummied,sessions_per_device, on='id', how = 'left')

In [42]:
users_dummied[['total_sessions','total_sessions_on_mac','total_sessions_on_windows','total_sessions_on_iphone','total_sessions_on_android']] = users_dummied[['total_sessions','total_sessions_on_mac','total_sessions_on_windows','total_sessions_on_iphone','total_sessions_on_android']].fillna(0)


In [43]:
users_dummied.diff_account_to_first_active = users_dummied.diff_account_to_first_active.replace(-1,0)

In [51]:
users_dummied['country_destination'] = users['country_destination']
users = users_dummied

In [52]:
users['no_age_entered'] = users.age.isnull()*1

In [53]:
users.age = users.age.fillna(users.age.mean())

In [54]:
with open('users_multiclass.pkl', 'wb') as picklefile: # wb: write, binary
    pickle.dump(users, picklefile) #dump data into pickle file

In [ ]:
users.columns

In [ ]:
plt.figure(figsize = (40,30))
ax = sns.heatmap(users.corr(),  linewidths=1.5, center = 0)

In [ ]:
import seaborn as sns